## 浙江省森林区域植被生长分析
利用  *<a href="https://engine-aiearth.aliyun.com/#/dataset" target="_blank">Modis MCD12Q1</a>* 地物分类数据产品和 *<a href="https://engine-aiearth.aliyun.com/#/dataset" target="_blank">MODIS MOD13Q1</a>* 16天标准植被指数产品。通过对 *2021* 年 *8* 月植被指数最大值与近 *3* 年同期指数 *8* 月最大值的 *3* 年均值进行对比，实现对浙江省森林区域植被的空间监测。

### 初始化环境

In [1]:
import aie

aie.Authenticate()
aie.Initialize()

### 定义矢量区域

In [2]:
region = aie.FeatureCollection('China_Province') \
            .filter(aie.Filter.eq('province', '浙江省')) 
            
map = aie.Map(
    center=region.getCenter(),
    height=800,
    zoom=5
)

vis_params = {
    'color': '#00FF00'
}
map.addLayer(
    region,
    vis_params,
    'region',
    bounds=region.getBounds()
)
map

Map(center=[29.162989375000002, 120.42838813], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoo…

### 获取森林区域掩膜
 引用 *<a href="https://engine-aiearth.aliyun.com/#/dataset" target="_blank">Modis MCD12Q1</a>* 地物分类数据产品，其中 *LC_Type1* 中数值 *1-5* 为不同类型的森林植被，通过 *<a href="https://engine-aiearth.aliyun.com/docs/page/api?d=044dbf" target="_blank">aie.Image.lt</a>* 实现 *2020* 年森林植被覆盖地区提取。并将提取到的数据进行地图可视化显示。

In [3]:
LC_Dataset = aie.ImageCollection('MODIS_MCD12Q1_006') \
                .filterDate('2020-05-01', '2020-05-31')
imgs = LC_Dataset.select(['LC_Type1']).first().clipToCollection(region)

forest = imgs.lt(aie.Image.constant(6))   # Modis MCD12Q1 1-5为不同类型的森林

vis_params = {
    'bands': 'LC_Type1',
    'min': 1,
    'max': 17,
    'palette': [ 
        '#05450a', '#086a10', '#54a708', '#78d203', '#009900', '#c6b044', 
        '#dcd159', '#dade48', '#fbff13', '#b6ff05', '#27ff87', '#c24f44', 
        '#a5a5a5', '#ff6d4c', '#69fff8', '#f9ffa4', '#1c0dff'
    ]
}

forest_vis = {
    'bands': 'LC_Type1',
    'min': 0,
    'max': 1,
    'palette': [ 
        '#ffffff', '#4fb104'
    ]
}

map.addLayer(
    imgs,
    vis_params,
    'LC_data',
    bounds=region.getBounds()
)

map.addLayer(
    forest,
    forest_vis,
    'Forest',
    bounds=region.getBounds()
)
map

Map(center=[29.162989375000002, 120.42838813], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoo…

### 植被生长对比
使用 *<a href="https://engine-aiearth.aliyun.com/#/dataset" target="_blank">MODIS MOD13Q1</a>* 16天标准植被指数产品，利用 *<a href="https://engine-aiearth.aliyun.com/docs/page/api?d=bffe02" target="_blank">aie.ImageCollection.max</a>* 获得 *2018、2019、2020年、2021* 年逐年 *8* 月 *NDVI* 最大值，并计算 *2018-2020* 年 *3* 年的均值（ *<a href="https://engine-aiearth.aliyun.com/docs/page/api?d=2a9c56" target="_blank">aie.ImageCollection.mean</a>* ），对比 *2021* 年与过去*3*年同期均值的比较，应用 *<a href="https://engine-aiearth.aliyun.com/docs/page/api?d=2ea198" target="_blank">updateMask</a>* 函数进行森林地区掩膜，确定 *2021* 年浙江森林植被生长状态。并将最终成果进行地图可视化显示。

In [4]:
# 指定检索数据集，可设置检索的时间范围
def MOD13Q_NDVI(start_time, end_time):
    VI_Dataset = aie.ImageCollection('MODIS_MOD13Q1_006') \
                    .filterDate(start_time, end_time) \
                    .select(['NDVI']) \
                    .max() \
                    .clipToCollection(region)
    return  VI_Dataset 

start_time = '08-01'
end_time = '08-30'

ndvi_2018 = MOD13Q_NDVI('2018-'+start_time, '2018-'+end_time) 
ndvi_2019 = MOD13Q_NDVI('2019-'+start_time, '2019-'+end_time)   
ndvi_2020 = MOD13Q_NDVI('2020-'+start_time, '2020-'+end_time)   
ndvi_2021 = MOD13Q_NDVI('2021-'+start_time, '2021-'+end_time)

ndvi_avg = aie.ImageCollection([ndvi_2018,ndvi_2019,ndvi_2020]).mean()
ndvi_dif_forest = (ndvi_2021.subtract(ndvi_avg)).updateMask(forest.eq(aie.Image.constant(1)))

In [5]:
ndvi_vis  = {
    'bands': 'NDVI',
    'min': 0,
    'max': 8000,
    'palette': [ 
        '#FFFFFF', '#CE7E45', '#DF923D', '#F1B555', '#FCD163', '#99B718',
        '#74A901', '#66A000', '#529400', '#3E8601', '#207401', '#056201',
        '#004C00', '#023B01', '#012E01', '#011D01', '#011301'
    ]
}

ndvi_dif_vis = {
    'min': -1000,
    'max': 1000,
    'palette': [
        '#d7191c', '#ffffff', '#008000'
    ]
}

map.addLayer( ndvi_avg, ndvi_vis,  'NDVI', bounds=region.getBounds() )

map.addLayer( ndvi_dif_forest, ndvi_dif_vis, 'NDVI_dif_forest',  bounds=region.getBounds())

map

Map(center=[29.162989375000002, 120.42838813], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoo…

*备注：本案例中仅取2020年森林分类成果作为掩膜文件，仅作为做算子应用介绍，数据成果合理性不做保证。*